In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


df_complet=pd.read_csv('Compilation webscrapping cosmetique.csv')
df = df_complet[['notes', 'commentaire']]

df.head()

,notes,commentaire
0,1,Conseils maquillages nuls ! un mascara qui fai...
1,1,Colis non encore reçu après deux semaines - il...
2,5,J'ai été merveilleusement accueillie et consei...
3,1,"Mais plus jamais, jamais , jamais!!! Sensé ouv..."
4,1,Toujours pas de remboursement après un mois et...


In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
df['notes'] = df['notes'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(df['commentaire'], df['notes'], test_size=0.2, random_state=42)

num_classes = len(df['notes'].cat.categories)

max_words = 1000  
max_len = 100 

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_text = tokenizer.texts_to_sequences(X_train)
X_test_text = tokenizer.texts_to_sequences(X_test)

train_text = pad_sequences(X_train_text, maxlen=max_len)
test_text = pad_sequences(X_test_text, maxlen=max_len)


In [45]:
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 50)           50000     
                                                                 
 spatial_dropout1d_4 (Spati  (None, 100, 50)           0         
 alDropout1D)                                                    
                                                                 
 lstm_4 (LSTM)               (None, 100)               60400     
                                                                 
 dense_4 (Dense)             (None, 5)                 505       
                                                                 
Total params: 110905 (433.22 KB)
Trainable params: 110905 (433.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train.cat.codes)
y_test_cat = to_categorical(y_test.cat.codes)


In [47]:
batch_size = 32
epochs = 5

model.fit(train_text, y_train_cat, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/5
124/124 [==============================] - 8s 48ms/step - loss: 0.9493 - accuracy: 0.6886 - val_loss: 0.6194 - val_accuracy: 0.8214
Epoch 2/5
124/124 [==============================] - 5s 41ms/step - loss: 0.6515 - accuracy: 0.7986 - val_loss: 0.5868 - val_accuracy: 0.8325
Epoch 3/5
124/124 [==============================] - 5s 41ms/step - loss: 0.5860 - accuracy: 0.8153 - val_loss: 0.5702 - val_accuracy: 0.8325
Epoch 4/5
124/124 [==============================] - 5s 42ms/step - loss: 0.5264 - accuracy: 0.8347 - val_loss: 0.5512 - val_accuracy: 0.8385
Epoch 5/5
124/124 [==============================] - 5s 41ms/step - loss: 0.4699 - accuracy: 0.8451 - val_loss: 0.5773 - val_accuracy: 0.8244


In [48]:
loss, accuracy = model.evaluate(test_text, y_test_cat)
print("L accuracy sur notre jeu de test est : ", accuracy)


39/39 [==============================] - 1s 15ms/step - loss: 0.6070 - accuracy: 0.8200
L accuracy sur notre jeu de test est :  0.8200161457061768


In [51]:
# Essai sur un très bon commentaire
comm1 = ["C'est un produit magique !"]

comm1_text = tokenizer.texts_to_sequences(comm1)
comm1_text = pad_sequences(comm1_text, maxlen=max_len)

prediction = model.predict(comm1_text)
predicted_class = np.argmax(prediction)
print("La note prédite est de : ", predicted_class+1)


1/1 [==============================] - 0s 26ms/step
La note prédite est de :  5


In [52]:
# Essai sur un très mauvais commentaire
comm2 = ["Horrible. Marque à fuir malgré les prix attractifs!"]

comm2_text = tokenizer.texts_to_sequences(comm2)
comm2_text = pad_sequences(comm2_text, maxlen=max_len)

prediction2 = model.predict(comm2_text)
predicted_class2 = np.argmax(prediction2)
print("La note prédite est de : ", predicted_class2+1)


1/1 [==============================] - 0s 26ms/step
La note prédite est de :  1


In [53]:
# Essai sur les commentaires du projet supply chaine
comm3 = ["C'est une mauvaise marque ; les produits ne sont pas qualitatifs"]

comm3_text = tokenizer.texts_to_sequences(comm3)
comm3_text = pad_sequences(comm3_text, maxlen=max_len)

prediction3 = model.predict(comm3_text)
predicted_class3 = np.argmax(prediction3)
print("La note prédite est de : ", predicted_class3+1)

1/1 [==============================] - 0s 27ms/step
La note prédite est de :  1


In [54]:
# Essai sur les commentaires du projet supply chaine
comm4 = ["J'apprécie beaucoup cette marque. Bien que le site soit en ligne, ils sont très réactifs."]

comm4_text = tokenizer.texts_to_sequences(comm4)
comm4_text = pad_sequences(comm4_text, maxlen=max_len)

prediction4 = model.predict(comm4_text)
predicted_class4 = np.argmax(prediction4)
print("La note prédite est de : ", predicted_class4+1)

1/1 [==============================] - 0s 25ms/step
La note prédite est de :  5


In [55]:
# Essai sur les commentaires du projet supply chaine
comm5 = ["C’est super, les prix sont tops "]

comm5_text = tokenizer.texts_to_sequences(comm5)
comm5_text = pad_sequences(comm5_text, maxlen=max_len)

prediction5 = model.predict(comm5_text)
predicted_class5 = np.argmax(prediction5)
print("La note prédite est de : ", predicted_class5+1)

1/1 [==============================] - 0s 27ms/step
La note prédite est de :  5


In [56]:
# Essai sur les commentaires du projet supply chaine
comm6 = ["Site nul ! Service client ne m’a même pas répondu"]

comm6_text = tokenizer.texts_to_sequences(comm6)
comm6_text = pad_sequences(comm6_text, maxlen=max_len)

prediction6 = model.predict(comm6_text)
predicted_class6 = np.argmax(prediction6)
print("La note prédite est de : ", predicted_class6+1)

1/1 [==============================] - 0s 27ms/step
La note prédite est de :  1


In [57]:
# Essai sur un nouveau commentaires pour projet supply chaine
comm7 = ["Je suis partagé. Les prix sont attractifs. Les produits s usent dans le temps et ne résistent pas."]

comm7_text = tokenizer.texts_to_sequences(comm7)
comm7_text = pad_sequences(comm7_text, maxlen=max_len)

prediction7 = model.predict(comm7_text)
predicted_class7 = np.argmax(prediction7)
print("La note prédite est de : ", predicted_class7+1)

1/1 [==============================] - 0s 27ms/step
La note prédite est de :  4
